In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from scipy import signal
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from glob import glob
import math
from sklearn.model_selection import train_test_split
from PIL.ImageOps import invert
from keras.utils import plot_model
from sklearn.metrics import classification_report
from gprGAN import gprGAN
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from keras.datasets import mnist
from keras.layers.merge import _Merge, multiply
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from functools import partial
import tensorflow as tf
from keras.utils import to_categorical

np.random.seed(1234)
tf.set_random_seed(1234)

os.environ["CUDA_VISIBLE_DEVICES"]="0"


from keras.backend.tensorflow_backend import set_session 
config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True 
# dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran) 
# (nothing gets printed in Jupyter, only if you run it standalone) 
sess = tf.Session(config=config)
set_session(sess) # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [2]:

noise = Input(shape=(100,))
label = Input(shape=(1,), dtype='int32')

label_embedding = Flatten()(Embedding(3, 100)(label))

model_input = multiply([noise, label_embedding])

out = Dense(128 * 8 * 8,
                        activation="relu")(model_input)
out = Reshape((8, 8, 128))(out)
        
out = BatchNormalization(momentum=0.8)(out)
out = UpSampling2D()(out)
out = Conv2D(256, kernel_size=5, padding="same")(out)
out = Activation("relu")(out)
        
out = BatchNormalization(momentum=0.8)(out)
out = UpSampling2D()(out)
out = Conv2D(128, kernel_size=5, padding="same")(out)
out = Activation("relu")(out)
out = BatchNormalization(momentum=0.8)(out)
out = UpSampling2D()(out)
out = Conv2D(64, kernel_size=5, padding="same")(out)
out = Activation("relu")(out)
        
out = BatchNormalization(momentum=0.8)(out)
out = UpSampling2D()(out)
out = Conv2D(32, kernel_size=5, padding="same")(out)
out = Activation("relu")(out)
        
out = BatchNormalization(momentum=0.8)(out)
out = UpSampling2D()(out)
out = Conv2D(16, kernel_size=5, padding="same")(out)
out = Activation("relu")(out)
        
out = BatchNormalization(momentum=0.8)(out)
out = Conv2D(1, kernel_size=3, padding='same')(out)
out = Activation("tanh")(out)

model = Model([noise, label], out)


model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       300         input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 100)          0           embedding_1[0][0]                
_____________________________________

In [4]:
img = Input((256, 256, 1))
out = Conv2D(16, kernel_size=5, strides=2, padding="same")(img)
out = LeakyReLU(alpha=0.3)(out)
out = Conv2D(16*2, kernel_size=5, strides=2, padding="same")(out)
out = LeakyReLU(alpha=0.3)(out)
out = Conv2D(16*4, kernel_size=5, strides=2, padding="same")(out)
out = LeakyReLU(alpha=0.3)(out)
out = Conv2D(16*8, kernel_size=5, strides=2, padding="same")(out)
out = LeakyReLU(alpha=0.3)(out)
out = Conv2D(16*16, kernel_size=5, strides=2, padding="same")(out)
out = LeakyReLU(alpha=0.3)(out)
out = Flatten()(out)

validity = Dense(1, activation=None)(out)
model = Model(img, validity)
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 16)      416       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 32)        12832     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 64)        0         
__________